# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

train = pd.read_csv("/content/drive/MyDrive/Final_Project_ML_OL/data/train.csv")
test  = pd.read_csv("/content/drive/MyDrive/Final_Project_ML_OL/data/test.csv")

Mounted at /content/drive


Cleanup

In [2]:
train.columns = train.columns.str.strip()
test.columns = test.columns.str.strip()

# Engineer Features

Handle Cabin

In [3]:
def split_cabin(df):
  df[['CabinDeck','CabinNum','CabinSide']] = df['Cabin'].str.split('/', expand=True)
  return df

train = split_cabin(train)
test = split_cabin(test)

Create group size

In [4]:
train['Group'] = train['PassengerId'].str.split('_').str[0]
test['Group'] = test['PassengerId'].str.split('_').str[0]

train['GroupSize'] = train.groupby('Group')['Group'].transform('count')
test['GroupSize'] = test.groupby('Group')['Group'].transform('count')

Total Spending

In [5]:
spend_cols = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
train['TotalSpend'] = train[spend_cols].sum(axis=1)
test['TotalSpend'] = test[spend_cols].sum(axis=1)

Feature Types

In [6]:
num_features = ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','TotalSpend','GroupSize']
cat_features = ['HomePlanet','Destination','CabinDeck','CabinSide','CryoSleep','VIP']


Preprocessing Pipeline

In [7]:
numeric_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='median')),
  ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='most_frequent')),
  ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
  transformers=[
    ('num', numeric_transformer, num_features),
    ('cat', categorical_transformer, cat_features)
  ]
)

Train/Validation Split

In [8]:
X = train[num_features + cat_features]
y = train['Transported']


X_train, X_val, y_train, y_val = train_test_split(
  X, y, test_size=0.2, random_state=42, stratify=y
)

X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(test[num_features + cat_features])

Save Datasets

In [9]:
import joblib
PREP_PATH = "/content/drive/MyDrive/Final_Project_ML_OL/data/preprocessor.joblib"
joblib.dump(preprocessor, PREP_PATH)
print("Saved preprocessor to:", PREP_PATH)

Saved preprocessor to: /content/drive/MyDrive/Final_Project_ML_OL/data/preprocessor.joblib


In [ ]:
np.save('/content/drive/MyDrive/Final_Project_ML_OL/data/X_train_processed.npy', X_train_processed)
np.save('/content/drive/MyDrive/Final_Project_ML_OL/data/X_val_processed.npy', X_val_processed)
np.save('/content/drive/MyDrive/Final_Project_ML_OL/data/X_test_processed.npy', X_test_processed)

y_train.to_csv('/content/drive/MyDrive/Final_Project_ML_OL/data/y_train.csv', index=False)
y_val.to_csv('/content/drive/MyDrive/Final_Project_ML_OL/data/y_val.csv', index=False)